Sort people into leagues for curling
KL 8/17/2016

This version randomizes the people first...seems better to do byLeague version (more people get first choice)

In [67]:
if True:
    %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [68]:
import pandas as pd
import numpy as np
import math
import csv
import os.path

#used this to step into the function and debug it, also need line with Tracer()() 
from IPython.core.debugger import Tracer 

Define some functions up top

In [70]:
def addOne(leagueDict,oneLeague,oneName):
    if len(leagueDict[oneLeague])==0: #nothing there yet
        leagueDict[oneLeague] = oneName
    else:     #need to append
        leagueDict[oneLeague] = leagueDict[oneLeague].append(oneName)
    return leagueDict

In [71]:
def addManager(r,manager):
    useRow = r.index[0]
    oneName = regSorted.loc[useRow,['lastname', 'firstname']] 
    oneName = oneName.to_frame().transpose() #need this format to make nice output
    leagueDict_open[manager] = oneName
    regSorted.loc[useRow,manager]=np.nan
    regTracking.loc[useRow,manager]='yes'

In [73]:
def addOpenLeague(idx,regSorted,regTracking,leagueDict_open,iteration,trackIssues):
    #person is interested in open leagues, oneLeague will be index in their next choice
    oneLeague = regSorted.loc[idx,openLeagues].dropna().idxmin()

    oneName = regSorted.loc[idx,['lastname', 'firstname']] 
    oneName = oneName.to_frame().transpose() #need this format to make nice output

    if len(leagueDict_open[oneLeague]) < (nLimit.loc[oneLeague,'nLimit']): #room in league
        leagueDict_open = addOne(leagueDict_open,oneLeague,oneName)
        #now that I have added the person to a league, change their priority to NaN
        regSorted.loc[idx,oneLeague] = np.nan
        regTracking.loc[idx,oneLeague] = 'yes'

    elif len(leagueDict_open[oneLeague]) == (nLimit.loc[oneLeague,'nLimit']):
        #league is full...does this person have another league listed?
        #first, set current to NaN
        regSorted.loc[idx,oneLeague] = np.nan
        regTracking.loc[idx,oneLeague] = 'no' #did not get in
        nextPriority = regSorted.loc[idx,openLeagues].dropna().min()
        if math.isnan(nextPriority)!=1: #have a number
            #go to the next league on their list
            nextLeague = regSorted.loc[idx,openLeagues].dropna().idxmin()
            #only add person is league has space
            if len(leagueDict_open[nextLeague]) < (nLimit.loc[nextLeague,'nLimit']):
                leagueDict_open = addOne(leagueDict_open,nextLeague,oneName)
                regSorted.loc[idx,nextLeague] = np.nan
                regTracking.loc[idx,nextLeague] = 'yes'
            elif (len(leagueDict_open[nextLeague]) == nLimit.loc[nextLeague,'nLimit']) & (iteration==1):
                regTracking.loc[idx,nextLeague]='no'
                trackIssues = addOne(trackIssues,'notInFirst',oneName)
            elif (len(leagueDict_open[nextLeague]) == nLimit.loc[nextLeague,'nLimit']) & (iteration==2):
                regTracking.loc[idx,nextLeague]='no'
                trackIssues = addOne(trackIssues,'notInSecond',oneName)
            elif (len(leagueDict_open[nextLeague]) == nLimit.loc[nextLeague,'nLimit']) & (iteration==3):
                regTracking.loc[idx,nextLeague]='no'
                trackIssues = addOne(trackIssues,'notInThird',oneName)
        elif math.isnan(nextPriority)==1:
            trackIssues=addOne(trackIssues,'noAdditionalChoice',oneName)
                           
    return regSorted, regTracking, leagueDict_open, trackIssues

Read in the data

In [74]:
#goal will be to use the version that comes out of the website
fName = 'TestPriority4.csv'
regInfo=pd.read_csv(fName)

#first, how many random numbers so I need?
nPeople = len(regInfo.index)
regInfo['randomNumCol'] = np.random.choice(range(nPeople), nPeople,replace = False)
##so now, moving back to the list of people, sort the list by the random number
regSorted = regInfo.sort_values(by = 'randomNumCol', ascending=True)
regSorted = regSorted.set_index(['randomNumCol'])

#copy so I can have an output tracking what was done, make after I put in league managers
regTracking = regSorted.copy(deep=True)

In [76]:
#now, make up a series of variables that are the names of the leagues, 
#can pull the leageu names from imported CSV file. They won't move around,
#so just index in, but don't need the Men's league in here
openLeagues = list(regInfo)[5:12]
openLeagues.remove('League_4_Men')
openLeagues

['League_1_Sunday_Open',
 'League_2_Monday_Open',
 'League_3_Monday_Night_Open',
 'League_Tuesday_Open',
 'League_5_Wednesday_Open',
 'League_6_Thursday_Open']

In [72]:
issueNames = ['notInFirst','notInSecond','notInThird','noAdditionalChoice']
#setup the dictionary to hold the issues
trackIssues = {}
for name in issueNames:
    trackIssues[name] = pd.DataFrame()

In [77]:
#setup the dictionary to hold the people information (start with openLeagues only)
forDictionary = openLeagues
#forDictionary.extend(otherLeagues)
leagueDict_open = {}
for name in forDictionary:
    leagueDict_open[name] = pd.DataFrame()

In [78]:
#set the limit for each league:
sevenTeams = 5 #change to 28 (remember league manager goes in first)
fourteenTeams = 56

d = {'League_1_Sunday_Open': sevenTeams,
     'League_2_Monday_Open': sevenTeams,
     'League_3_Monday_Night_Open': fourteenTeams,
     'League_Tuesday_Open': sevenTeams,
     'League_5_Wednesday_Open': sevenTeams,
     'League_6_Thursday_Open': sevenTeams,}

nLimit = pd.DataFrame({'nLimit' : d})
del d

In [79]:
#put the league managers into their league and set their the choice for that league to NaN
manager = 'League_1_Sunday_Open'
r = regSorted.loc[(regSorted['lastname']=='Fragaszy') & (regSorted['firstname']=='Bill')]
addManager(r,manager)

manager = 'League_2_Monday_Open'
r = regSorted.loc[(regSorted['lastname']=='Brown') & (regSorted['firstname']=='Peter')]
addManager(r,manager)

manager = 'League_3_Monday_Night_Open'
r = regSorted.loc[(regSorted['lastname']=='DiMassa') & (regSorted['firstname']=='Diane')]
addManager(r,manager)

manager = 'League_Tuesday_Open'
r = regSorted.loc[(regSorted['lastname']=='Thomson') & (regSorted['firstname']=='Kenny')]
addManager(r,manager)

manager = 'League_5_Wednesday_Open'
r = regSorted.loc[(regSorted['lastname']=='Murphy') & (regSorted['firstname']=='Katie')]
addManager(r,manager)

manager = 'League_6_Thursday_Open'
r = regSorted.loc[(regSorted['lastname']=='Gallagher') & (regSorted['firstname']=='Bill')]
addManager(r,manager)

In [80]:
#go forwards through the list to start
iteration = 1
for idx in (range(0,nPeople)):
    #only do something if a person is still looking for more leagues...
    rt = regTracking.loc[idx,openLeagues]
    cHave = len(rt[rt=='yes'])
    del rt
    if cHave < regTracking.loc[idx,'numberLeagues']: #person wants more 
        onePriority = regSorted.loc[idx,openLeagues].dropna().min() #any more things ranked?   
        if math.isnan(float(onePriority))==False:
            regSorted,regTracking,leagueDict_open,trackIssues = addOpenLeague(idx,regSorted,regTracking,leagueDict_open,iteration,trackIssues)

In [81]:
#now go backwards through the list to fill the other spots
iteration = 2
for idx in reversed(range(0,nPeople)):
#for idx in range(0,15):
    #only do something if a person is still looking for more leagues...
    rt = regTracking.loc[idx,openLeagues]
    cHave = len(rt[rt=='yes'])
    del rt
    if cHave < regTracking.loc[idx,'numberLeagues']: #person wants more 
        onePriority = regSorted.loc[idx,openLeagues].dropna().min() #any more things ranked?   
        if math.isnan(float(onePriority))==False:
            regSorted,regTracking,leagueDict_open,trackIssues = addOpenLeague(idx,regSorted,regTracking,leagueDict_open,iteration,trackIssues)
        

In [82]:
#go forwards again...stop here...though I could go backwards again if there are people who want to curl 4x @ 4pm
iteraction = 3
for idx in (range(0,nPeople)):
    #only do something if a person is still looking for more leagues...
    rt = regTracking.loc[idx,openLeagues]
    cHave = len(rt[rt=='yes'])
    del rt
    if cHave < regTracking.loc[idx,'numberLeagues']: #person wants more 
        onePriority = regSorted.loc[idx,openLeagues].dropna().min() #any more things ranked?   
        if math.isnan(float(onePriority))==False:
            regSorted,regTracking,leagueDict_open,trackIssues = addOpenLeague(idx,regSorted,regTracking,leagueDict_open,iteration,trackIssues)

In [83]:
#make a new dictionary for other leagues...concat with openLeagues when done
# do doubles and competitive league by hand...
otherLeagues = ['League_4_Men', 'League_7_Ladies' ,'League_Scrod']

leagueDict_other = {}
for name in otherLeagues:
    leagueDict_other[name] = pd.DataFrame()

In [84]:
#only have to go through once
for idx in range(0,nPeople):
    for oL in otherLeagues:
    
        oneName = regSorted.loc[idx,['lastname', 'firstname']] 
        oneName = oneName.to_frame().transpose() #need this format to make nice output
        if regSorted.loc[idx,oL]=='Yes': #careful with trailing spaces
            leagueDict_other = addOne(leagueDict_other,oL,oneName)
            #now that I have added the person to a league, change their priority to NaN
            regTracking.loc[idx,oL] = 'done'
            regSorted.loc[idx,oL] = np.nan

In [85]:
#combine the two dictionaries
leagueDict_all = leagueDict_open.copy()
leagueDict_all.update(leagueDict_other)

#combine the list of names
allLeagues = list(openLeagues)
allLeagues.extend(otherLeagues)

In [86]:
#finally, export the results (CAREFUL...will delete existing files)
for fn in allLeagues:
    fName = '%s.csv' % fn
    #check if the file exists...delete if it does
    if os.path.isfile(fName):
        os.remove(fName)
        
    leagueDict_all[fn].to_csv(fName)

ran multiple times and saved the output

In [88]:
#count up number of issues
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'

notInFirst :  37  people in this category
notInSecond :  15  people in this category
notInThird :  0  people in this category
noAdditionalChoice :  80  people in this category


In [66]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'

notInFirst :  38  people in this category
notInSecond :  16  people in this category
notInThird :  0  people in this category
noAdditionalChoice :  79  people in this category


In [44]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'

notInFirst :  38  people in this category
notInSecond :  15  people in this category
notInThird :  0  people in this category
noAdditionalChoice :  77  people in this category


In [22]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'

notInFirst :  36  people in here
notInSecond :  16  people in here
notInThird :  0  people in here
noAdditionalChoice :  79  people in here
